# Obtem todas as senteças que os usuarios classificados produziram

In [143]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import yaml

filepaths = yaml.safe_load(open("../config/filepaths.yaml"))

In [46]:
episodes = pd.read_csv(filepaths['interview_original']+'episodes.csv')
episodes.episode_date = episodes.episode_date.apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d"))
episodes

,id,program,title,episode_date
0,98814,Morning Edition,Senate Ushers In New Year With 'Fiscal Cliff' ...,2013-01-01
1,98824,Morning Edition,Cheap Bubbly Or Expensive Sparkling Wine? Look...,2012-12-31
2,98821,Morning Edition,U.S. Gas Prices Reach Record Level In 2012,2013-01-01
3,98806,Morning Edition,House Approves 'Fiscal Cliff' Measure,2013-01-02
4,98823,Morning Edition,The Fiscal Cliff: A Love Story,2012-12-31
...,...,...,...,...
105843,98839,All Things Considered,"With Malaysia Airlines Crash, A Loss For AIDS ...",2014-07-19
105844,98840,All Things Considered,"The Israeli-Palestinian Conflict, Remixed And ...",2014-07-19
105845,98842,All Things Considered,"As Superheroes Go Mainstream, Comic Cons Get C...",2014-07-19
105846,98833,All Things Considered,"Despite California's Drought, Taps Still Flowi...",2014-07-20


In [13]:
episodes = episodes[episodes.episode_date>datetime.datetime.strptime('2010',"%Y")]

In [14]:
interview = pd.read_csv(filepaths['interview_original']+'utterances-2sp.csv')
interview.head()

,episode,episode_order,turn_order,speaker_order,host_id,is_host,utterance
0,1,1,0,0,0,True,The impeachment inquiry picks up tomorrow wher...
1,1,1,1,0,0,True,"Just this morning, the lawyer for the whistleb..."
2,1,1,2,0,0,True,There's are a lot of moving parts.
3,1,1,3,0,0,True,"Fortunately, NPR's Mara Liasson is here to help."
4,1,2,0,0,0,True,Good morning.


In [15]:
interview = interview[
    (interview.is_host==False)
    & (interview.episode.isin(episodes.id.unique()))
]
len(interview)

443184

In [16]:
# exclui episodeos em que depois de selecionar somente o entrevistado (acima) 
# o speaker order é duplicado considerando que somente deveriam existir 2 pessoas na conversa

more_than_one = interview[['episode','speaker_order']].drop_duplicates()['episode'][interview[['episode','speaker_order']].drop_duplicates()['episode'].duplicated()]
interview = interview[~ interview.episode.isin(more_than_one)]
len(interview)

438458

In [17]:
all_episodes = pd.read_csv(filepaths['interview_original']+'utterances.csv')
all_episodes.head()

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."


In [18]:
speakers = all_episodes[all_episodes.episode.isin(interview.episode.unique())][['episode','speaker']].drop_duplicates()
speakers.speaker = speakers.speaker.str.lower()
speakers = speakers[~ speakers.speaker.str.contains('_no_speaker')]

speakers.groupby('episode').count().query('speaker > 2')

,speaker
episode,


In [19]:
speakers[speakers.speaker.str.contains('host')].episode.nunique()

13423

In [20]:
guests = speakers[~speakers.speaker.str.contains('host')]
guests.episode.nunique()

13416

In [21]:
guests.groupby('episode').count().query('speaker > 1')

,speaker
episode,


In [22]:
print(len(interview))
sentences = interview.merge(guests)[['episode','episode_order','turn_order','utterance','speaker']]
print(len(sentences))

438458
438249


In [23]:
sentences['utterance_id'] = sentences[['episode','episode_order','turn_order']].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
sentences = sentences.drop(columns=['episode','episode_order','turn_order'])
sentences.head()

,utterance,speaker,utterance_id
0,"Good morning, Lulu.","mara liasson, byline",1_3_0
1,"Well, the latest is that the lawyer for the fi...","mara liasson, byline",1_5_0
2,The first whistleblower only had second and th...,"mara liasson, byline",1_5_1
3,We also know that there are subpoenas for Whit...,"mara liasson, byline",1_5_2
4,We don't know how cooperative the administrati...,"mara liasson, byline",1_5_3


In [24]:
sentences

,utterance,speaker,utterance_id
0,"Good morning, Lulu.","mara liasson, byline",1_3_0
1,"Well, the latest is that the lawyer for the fi...","mara liasson, byline",1_5_0
2,The first whistleblower only had second and th...,"mara liasson, byline",1_5_1
3,We also know that there are subpoenas for Whit...,"mara liasson, byline",1_5_2
4,We don't know how cooperative the administrati...,"mara liasson, byline",1_5_3
...,...,...,...
438244,Thank you so much.,ms. caroline herring (singer),122299_46_0
438245,"(Singing) I had a few more questions, I never ...",ms. caroline herring (singer),122299_49_0
438246,"You were feeling downhearted, the last time we...",ms. caroline herring (singer),122299_49_1
438247,"With a shuck of the white hair, life has chang...",ms. caroline herring (singer),122299_49_2


In [26]:
classifictions = pd.read_csv(filepaths['interview_classifications']+'all.csv')
classifictions.head()

,speaker,gender,race,speaker_original
0,andrew chamberlain,f,white,andrew chamberlain
1,reese witherspoon,f,white,reese witherspoon
2,karoun demirjian,f,white,karoun demirjian
3,rachel held evans,f,white,rachel held evans
4,jennifer pahlka,f,white,jennifer pahlka


In [33]:
sentences = sentences.merge(classifictions, left_on='speaker',right_on='speaker_original',how='right',suffixes=('_sentences','')).drop(columns=['speaker_sentences'])

In [36]:
sentences.groupby(['race','gender']).count()

utterance  utterance_id  speaker  speaker_original
race  gender                                                    
black f            1698          1698     1698              1698
      m            2001          2001     2001              2001
white f            1729          1729     1729              1729
      m            1499          1499     1499              1499

#### Salva na formatação desejada

In [66]:
for spkr in sentences.speaker.unique():
    df = sentences[sentences.speaker==spkr]
    spkr_data = classifictions[classifictions.speaker==spkr].iloc[0]
    df[['utterance']].rename(columns={'utterance':'text'}).to_csv(
        filepaths['interview_sentences']+{}-{}-{}'.format(spkr_data.race,spkr_data.gender,spkr_data.speaker),
        index=False
    )